# Testing Spherical Harmonics and Other Components

In [1]:
# This is just giving me incorrect results, I'm pretty sure
using BenchmarkTools
using Test
using SphericalHarmonics

include("Spherical.jl");

In [15]:
include("utils.jl");

## Previous Tests

In [7]:
@testset "Wikipedia Clebsch-Gordan Coefficients" begin
    # Some examples from Wikipedia
    # TODO Get SymPy.jl comparison to work

    # cg(j1, m1, j2, m2, J, M) = <j1 m1 j2 m2; J M>
    @test cg(1, 1, 1, -1, 2, 0) == √(1 / 6)
    @test cg(1, 0, 1, 0, 2, 0) == √(2 / 3)
    @test cg(1, 0, 1, 0, 0, 0) == -√(1 / 3)
    @test cg(1, 0, 1, 0, 1, 0) == 0

    @test cg(2, 1, 1, 0, 3, 1) == √(8 / 15)
end;

Test Summary:               | Pass  Total  Time
Clebsch-Gordan Coefficients |    5      5  0.0s


In [49]:
@testset "Spherical Harmonics" begin
    Y00(θ, ϕ) = 0.5 * sqrt(1/π)
    Y1m1(θ, ϕ) = 0.5 * sqrt(3/(2*π))*sin(θ)*exp(-im*ϕ)
    Y10(θ, ϕ) = 0.5 * sqrt(3/π)*cos(θ)
    Y11(θ, ϕ) = -0.5 * sqrt(3/(2*π))*sin(θ)*exp(im*ϕ)

    # Real version
    function Y1mR(θ, ϕ, m::Integer)
        ans = Y10(θ, ϕ) |> real
        if m > 0
            # TODO Check proper minus signs, make sure convention is consistent
            # Seems to be a minus sign off?
            ans = -(Y1m1(θ, ϕ) + (-1)^m * Y11(θ, ϕ)) / √2 |> real
        elseif m < 0
            ans = -im * (Y1m1(θ, ϕ) - (-1)^m * Y11(θ, ϕ)) / √2 |> real
        end

        return ans
    end

    n_samples = 10
    for n = 1:n_samples
        θ = rand() * π
        ϕ = (rand()-0.5) * 2*π
        YsR = computeYlm(θ, ϕ; lmax=1, SHType=SphericalHarmonics.RealHarmonics())

        @test YsR[(0, 0)] ≈ Y00(θ, ϕ)
        @test YsR[(1, -1)] ≈ Y1mR(θ, ϕ, -1)
        @test YsR[(1, 1)] ≈ Y1mR(θ, ϕ, 1)
        @test YsR[(1, 0)] ≈ Y1mR(θ, ϕ, 0)
        #@test Ys[(1, -1)] ≈ Y1m1(θ, ϕ)
    end
end

Test Summary:       | Pass  Total  Time
Spherical Harmonics |   40     40  0.0s


Test.DefaultTestSet("Spherical Harmonics", Any[], 40, false, false, true, 1.672838802067e9, 1.672838802094e9)

## Testing Importance of Change of Basis

There are two equivalent change of basis formulae for converting complex spherical harmonics $Y_{\ell}^{m}: S^2 \mapsto \mathbb{C}$ to real spherical harmonics $Y_{\ell m}: S^2 \mapsto \mathbb{R}$. (Notice different subscript and superscript in notation.)
$$
Y_{\ell m}= \begin{cases}\frac{i}{\sqrt{2}}\left(Y_{\ell}^{-|m|}-(-1)^m Y_{\ell}^{|m|}\right) & \text { if } m<0 \\ Y_{\ell}^0 & \text { if } m=0 \\ \frac{1}{\sqrt{2}}\left(Y_{\ell}^{-|m|}+(-1)^m Y_{\ell}^{|m|}\right) & \text { if } m>0\end{cases}
\qquad
Y_{\ell m}= \begin{cases}\sqrt{2}(-1)^m \Im\left[Y_{\ell}^{|m|}\right] & \text { if } m<0 \\ Y_{\ell}^0 & \text { if } m=0 \\ \sqrt{2}(-1)^m \Re\left[Y_{\ell}^m\right] & \text { if } m>0\end{cases}
$$

The real spherical harmonics are meant to be a complete set of orthonormal functions too.

### Testing Minus Signs

Have determined that `SphericalHarmonics.jl` does **not** include the "Condon-Shortley" phase factor $(-1)^m$ above (most easily visible in the right-hand definition). (The choice comes down to whether or not $(-1)^m$ is given to the Legendre polynomial instead, and shouldn't make a meaningful difference.)

In [41]:
function Ys_c2r(Ys, ℓ, m)
    Y_r = Ys[(ℓ, 0)] |> real
    if m < 0
        Y_r = 1im * (Ys[(ℓ, -abs(m))] - (-1)^m * Ys[(ℓ, abs(m))]) / √2
    elseif m > 0
        Y_r = (Ys[(ℓ, -abs(m))] + (-1)^m * Ys[(ℓ, abs(m))]) / √2
    end
    Y_r
end

# Test passing means that SphericalHarmonics.jl does not include (-1)^m factor like Wikipedia definitions.
@testset "Check Minus Sign Convention" begin
    @variables θ::Real, ϕ::Real
    ℓ_max = 3
    Ys_real = computeYlm(θ, ϕ; lmax=ℓ_max, SHType=SphericalHarmonics.RealHarmonics())
    Ys_complex = computeYlm(θ, ϕ; lmax=ℓ_max)
    
    for ℓ in 0:ℓ_max
        for m in -ℓ:ℓ
            @test (-1)^m * Ys_c2r(Ys_complex, ℓ, m) - Ys_real[(ℓ, m)] |> real |> eval ≈ 0
        end
    end
end

Test Summary:               | Pass  Total  Time
Check Minus Sign Convention |   16     16  1.2s


Test.DefaultTestSet("Check Minus Sign Convention", Any[], 16, false, false, true, 1.67361158872e9, 1.673611589877e9)

### Test Algebraic Relations Between CG Coefficients

In [25]:
ℓi, ℓf, ℓo = 3, 2, 1
ai, af, ao = basis_rotation.((ℓi, ℓf, ℓo))
#generate_CG_matrices(ℓi, ℓf, ℓo)

(ComplexF64[-0.7071067811865475 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.7071067811865475 + 0.0im; 0.0 + 0.0im -0.7071067811865475 + 0.0im … 0.7071067811865475 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.7071067811865475im … 0.0 + 0.7071067811865475im 0.0 + 0.0im; 0.0 + 0.7071067811865475im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.7071067811865475im], ComplexF64[-0.0 - 0.7071067811865475im -0.0 + 0.0im … -0.0 + 0.0im 0.0 + 0.7071067811865475im; -0.0 + 0.0im -0.0 - 0.7071067811865475im … 0.0 + 0.7071067811865475im -0.0 + 0.0im; … ; -0.0 + 0.0im -0.7071067811865475 + 0.0im … -0.7071067811865475 + 0.0im -0.0 + 0.0im; -0.7071067811865475 + 0.0im -0.0 + 0.0im … -0.0 + 0.0im -0.7071067811865475 + 0.0im], ComplexF64[0.7071067811865475 + 0.0im 0.0 + 0.0im -0.7071067811865475 - 0.0im; 0.0 + 0.0im 0.0 - 1.0im 0.0 + 0.0im; 0.0 - 0.7071067811865475im 0.0 + 0.0im 0.0 - 0.7071067811865475im])

### Test Orthogonality